In [1]:
from future.backports.datetime import datetime
from misc.misc import run_in_thread
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TKAgg')
from pianoq.lab.mplc.mask_aligner import MPLCAligner
from pianoq.lab.pco_camera import PCOCamera
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mplc_device import MPLCDevice
import os 
import datetime
import time 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import glob 
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
import numpy as np 
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_10_14_00_34_old.masks"
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_12_19_58_diffuser_d1d.masks"

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
phases = np.random.uniform(0, 2*np.pi, size=50)
masks = add_phase_input_spots(masks, phases)
mplc.load_masks(masks, linear_tilts=True, plane_10_tilts=35)
mplc.restore_location()

Got MPLC


In [3]:
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

# mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
# myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
# print('Got Thorlabs motors')

Got Zaber motors


In [4]:
# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)

In [44]:
mxs.move_absolute(12.99)
mys.move_absolute(6.79)
print(mxs.get_position())
print(mys.get_position())

12.990004500000001
6.7899915


In [55]:
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread

integration_time = 1
start_x = 12.65
end_x = 13.2
start_y = 5
end_y = 6.8

mxs.wait_after_move = 0.1
mys.wait_after_move = 0.1

resolution = 1 
x_pixels = 22 // resolution
y_pixels = 72 // resolution
pixel_size_x = 0.025 * resolution
pixel_size_y = 0.025 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_signal_path = fr'{dir_path}\{timestamp}_signal_5_spots_tilt_35_deadzone_450_fine.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_signal_path)   
_ = scanner.scan(x_motor=mxs, y_motor=mys, ph=tt)

Moving to starting position...
starting scan
nan issue in timetagger
dur: 0. pix: 0, 0. Singles1: 594. Singles2: 988. Coincidence: 0.00.
dur: 2. pix: 0, 1. Singles1: 740. Singles2: 1032. Coincidence: 0.00.
dur: 4. pix: 0, 2. Singles1: 810. Singles2: 1058. Coincidence: 0.00.
dur: 5. pix: 0, 3. Singles1: 829. Singles2: 1055. Coincidence: 0.00.
dur: 6. pix: 0, 4. Singles1: 912. Singles2: 1157. Coincidence: 0.00.
dur: 8. pix: 0, 5. Singles1: 1027. Singles2: 1182. Coincidence: 0.00.
dur: 9. pix: 0, 6. Singles1: 1152. Singles2: 1256. Coincidence: 0.00.
dur: 10. pix: 0, 7. Singles1: 1249. Singles2: 1266. Coincidence: 0.00.
dur: 12. pix: 0, 8. Singles1: 1295. Singles2: 1262. Coincidence: 0.00.
dur: 13. pix: 0, 9. Singles1: 1191. Singles2: 1270. Coincidence: 0.00.
dur: 15. pix: 0, 10. Singles1: 1179. Singles2: 1269. Coincidence: 1.00.
dur: 16. pix: 0, 11. Singles1: 1040. Singles2: 1222. Coincidence: 0.00.
dur: 17. pix: 0, 12. Singles1: 1009. Singles2: 1068. Coincidence: 0.00.
dur: 19. pix: 0, 1